# Stock Aggregation Use Case

### 03.03.Setup the local database

Create sample data in the local warehouse database. This will be created for 3 warehouses, namely NewYork, Los Angeles and London. For convenience sake, data for all 3 warehouses will be stored in the same database. In a realworld situation, each warehouse will have its own database

In [1]:
#Create a database connection & cursor

import mariadb

#Connect to warehouse_stock database
local_db_conn = mariadb.connect(
                user="spark",
                password="spark",
                host="127.0.0.1",
                port=3306,
                database="warehouse_stock",
                autocommit=True
            )

local_cursor = local_db_conn.cursor()


In [2]:
#Delete existing data if needed. Execute code if needed to reinitialize database
local_cursor.execute("DELETE FROM warehouse_stock.item_stock")


In [3]:
import random
import datetime

def generate_data(generate_dates, warehouse_id):
    print("Generating sample data for warehouse : ", warehouse_id)

    #Create sample list of products and their unit values
    item_list = {
        "Tape Dispenser" : 5.99,
        "Pencil Sharpener" : 10.00,
        "Labeling Machine" : 25.00,
        "Calculator" : 14.99,
        "Scissors" : 7.99,
        "Sticky Notes" : 2.00,
        "Notebook" : 2.50,
        "Clipboard" : 12.00,
        "Folder" : 1.00,
        "Pencil Box" : 2.99
    }


    #Loop for each date
    for gen_date in generate_dates:
        print("Generating data for date : ", gen_date)

        for item, unit_value in item_list.items():
            #Generate random values for opening stock, receipts and issues
            opening_stock = random.randint(1,100)
            receipts=random.randint(1,50)
            issues=random.randint(1, opening_stock+receipts)

            insert_sql = f"""
                INSERT INTO `warehouse_stock`.`item_stock` 
                (`STOCK_DATE`, `WAREHOUSE_ID`,`ITEM_NAME`,
                    `OPENING_STOCK`,`RECEIPTS`,`ISSUES`,`UNIT_VALUE` )
                VALUES ( '{gen_date}','{warehouse_id}','{item}',
                    {opening_stock},{receipts},{issues},{unit_value} )
            """
            local_cursor.execute(insert_sql)


#Generate last 3 dates to push stock data for
generate_dates = [
    (datetime.datetime.today()-datetime.timedelta(2)).strftime("%Y-%m-%d"),
    (datetime.datetime.today()-datetime.timedelta(1)).strftime("%Y-%m-%d"),
    datetime.datetime.today().strftime("%Y-%m-%d")
    ]
    
#Generate for 3 warehouses
generate_data(generate_dates,"NewYork")
generate_data(generate_dates,"LosAngeles")
generate_data(generate_dates,"London")

#Get count of records
local_cursor.execute("""SELECT `WAREHOUSE_ID`, count(*) AS RECS
                        FROM `warehouse_stock`.`item_stock`
                        GROUP BY `WAREHOUSE_ID`""")

print("\nRecords created:\n---------------------------")
for warehouse_id, recs in local_cursor:
    print( warehouse_id, " : ", recs)

Generating sample data for warehouse :  NewYork
Generating data for date :  2025-12-29
Generating data for date :  2025-12-30
Generating data for date :  2025-12-31
Generating sample data for warehouse :  LosAngeles
Generating data for date :  2025-12-29
Generating data for date :  2025-12-30
Generating data for date :  2025-12-31
Generating sample data for warehouse :  London
Generating data for date :  2025-12-29
Generating data for date :  2025-12-30
Generating data for date :  2025-12-31

Records created:
---------------------------
London  :  30
LosAngeles  :  30
NewYork  :  30


### 03.04. Upload stock to the central store

Each warehouse uploads to a central store, mostly an S3 folder or a HDFS folder. We simulate this folder with a local directory. Data is stored as distributed files, partitioned by date and warehouse ID

In [4]:
from pyspark.sql import SparkSession

#We will reuse the local_db_conn already created. In real implementation, 
#this will be a separate script, so database connected need to be created.

#create spark session for Windows
local_spark = SparkSession\
            .builder\
            .appName("StockUploaderJob")\
            .config("spark.sql.shuffle.partitions", 2)\
            .config("spark.default.parallelism", 2)\
            .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True)\
            .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version","2")\
            .config("spark.jars", "jars/mysql-connector-j-8.4.0.jar") \
            .config("spark.driver.extraClassPath","jars/mysql-connector-j-8.4.0.jar") \
            .master("local[2]")\
            .getOrCreate()

#Set logging level to warn
local_spark.sparkContext.setLogLevel("WARN")


In [5]:
def upload_stock(start_date,end_date,warehouse_id):
    print(f"""Uploading stock for dates {start_date} to {end_date} for {warehouse_id}""")

    #find min and max bounds for the parallel DB query
    local_cursor.execute(f"""
        SELECT min(`ID`) as MIN_ID, max(`ID`) as MAX_ID 
        FROM `warehouse_stock`.`item_stock`
        WHERE `STOCK_DATE` BETWEEN '{start_date}' AND '{end_date}'
            AND `WAREHOUSE_ID` = '{warehouse_id}'
    """)

    min_bounds=0
    max_bounds=0
    for min_id, max_id in local_cursor:
        min_bounds=min_id
        max_bounds=max_id
    print("Query bounds are : ", min_id, max_id)

    stock_query=f"""
        SELECT `ID`, date_format(`STOCK_DATE`,'%Y-%m-%d') as STOCK_DATE, `WAREHOUSE_ID`, 
            `ITEM_NAME`, `OPENING_STOCK`, `RECEIPTS`, `ISSUES`, `UNIT_VALUE`
        FROM `warehouse_stock`.`item_stock`
        WHERE `STOCK_DATE` BETWEEN '{start_date}' AND '{end_date}'
            AND `WAREHOUSE_ID` = '{warehouse_id}'
    """

    #Using mysql since there is a bug in mariadb connector for spark
    #Using the workaround : https://issues.apache.org/jira/browse/SPARK-25013
    stock_df = local_spark.read\
                .format("jdbc")\
                .option("url", "jdbc:mysql://localhost:3306/warehouse_stock")\
                .option("dbtable", "( " + stock_query + " ) as tmpStock")\
                .option("user", "spark")\
                .option("password", "spark")\
                .option("partitionColumn","ID")\
                .option("lowerBound", min_bounds)\
                .option("upperBound",max_bounds + 1)\
                .option("numPartitions",2)\
                .load()
    
    stock_df.show(3)

    #Save the records to the distributed file system in the Central data center
    #Records are partitioned by stock_date and warehouse ID
    stock_df.write\
        .mode("append")\
        .partitionBy("STOCK_DATE","WAREHOUSE_ID")\
        .parquet("raw_data/")
    
start_date = generate_dates[0]
end_date = generate_dates[2]

#Generate for 3 warehouses
upload_stock(start_date,end_date,"NewYork")
upload_stock(start_date,end_date,"LosAngeles")
upload_stock(start_date,end_date,"London")


Uploading stock for dates 2025-12-29 to 2025-12-31 for NewYork
Query bounds are :  1 30
+---+----------+------------+----------------+-------------+--------+------+----------+
| ID|STOCK_DATE|WAREHOUSE_ID|       ITEM_NAME|OPENING_STOCK|RECEIPTS|ISSUES|UNIT_VALUE|
+---+----------+------------+----------------+-------------+--------+------+----------+
|  1|2025-12-29|     NewYork|  Tape Dispenser|            6|      42|    29|      5.99|
|  2|2025-12-29|     NewYork|Pencil Sharpener|           34|      15|    21|     10.00|
|  3|2025-12-29|     NewYork|Labeling Machine|           94|       4|    14|     25.00|
+---+----------+------------+----------------+-------------+--------+------+----------+
only showing top 3 rows

Uploading stock for dates 2025-12-29 to 2025-12-31 for LosAngeles
Query bounds are :  31 60
+---+----------+------------+----------------+-------------+--------+------+----------+
| ID|STOCK_DATE|WAREHOUSE_ID|       ITEM_NAME|OPENING_STOCK|RECEIPTS|ISSUES|UNIT_VALUE|
+--

### 03.05. Aggregating stock across warehouses

Aggregate total stock by item across warehouses and save them to a central mysql database

In [6]:
#Connect to global_stock database
global_db_conn = mariadb.connect(
                user="spark",
                password="spark",
                host="127.0.0.1",
                port=3306,
                database="global_stock",
                autocommit=True
            )

global_cursor = global_db_conn.cursor()

#create spark session for aggregating
global_spark = SparkSession\
            .builder\
            .appName("GlobalAggregatorJob")\
            .config("spark.sql.shuffle.partitions", 2)\
            .config("spark.default.parallelism", 2)\
            .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True)\
            .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version","2")\
            .config("spark.jars", "jars/mysql-connector-j-8.4.0.jar") \
            .config("spark.driver.extraClassPath","jars/mysql-connector-j-8.4.0.jar") \
            .master("local[2]")\
            .getOrCreate()

#Set logging level to warn
global_spark.sparkContext.setLogLevel("WARN")


In [7]:
#Delete existing data if needed. Execute code if needed to reinitialize database
global_cursor.execute("DELETE FROM global_stock.item_stock")


In [8]:
#Read the global stock data from the central file system
#this code reads files for all the dates. The can also be done one date at a time
#by appending the date directory to the URL
global_stock_df = global_spark.read.\
                    parquet("raw_data/")

#Create a view for using SparkSQL
global_stock_df.createOrReplaceTempView("GLOBAL_STOCK")

#Create a summary dataframe based on a summary query
summary_df = global_spark.sql("""
                    SELECT STOCK_DATE, ITEM_NAME,
                            COUNT(*) as TOTAL_REC,
                            SUM(OPENING_STOCK) as OPENING_STOCK, 
                            SUM(RECEIPTS) as RECEIPTS, 
                            SUM(ISSUES) as ISSUES,
                            SUM( OPENING_STOCK + RECEIPTS - ISSUES) as CLOSING_STOCK,
                            SUM( (OPENING_STOCK + RECEIPTS - ISSUES) * UNIT_VALUE ) as CLOSING_VALUE 
                            FROM GLOBAL_STOCK 
                            GROUP BY STOCK_DATE, ITEM_NAME
                        """)
print("Global Stock Summary")
summary_df.show(5)

#Write usmmary to MariaDB table
summary_df.write\
    .mode("append")\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost:3306/global_stock")\
    .option("dbtable", "global_stock.item_stock")\
    .option("user", "spark")\
    .option("password", "spark")\
    .save()

#Pruning: After processsing is over, delete the files in raw_data

Global Stock Summary
+----------+----------------+---------+-------------+--------+------+-------------+-------------+
|STOCK_DATE|       ITEM_NAME|TOTAL_REC|OPENING_STOCK|RECEIPTS|ISSUES|CLOSING_STOCK|CLOSING_VALUE|
+----------+----------------+---------+-------------+--------+------+-------------+-------------+
|2025-12-31|  Tape Dispenser|        3|          216|      65|   134|          147|       880.53|
|2025-12-31|Pencil Sharpener|        3|           66|      40|    48|           58|       580.00|
|2025-12-31|Labeling Machine|        3|          153|      63|   155|           61|      1525.00|
|2025-12-31|        Scissors|        3|          173|      68|   162|           79|       631.21|
|2025-12-31|    Sticky Notes|        3|          136|     101|   175|           62|       124.00|
+----------+----------------+---------+-------------+--------+------+-------------+-------------+
only showing top 5 rows



In [9]:
#Check if summary is computed correctly
#Query local database for individual records
local_check_cursor=local_db_conn.cursor()
local_check_cursor.execute(
    f"""SELECT `WAREHOUSE_ID`, `OPENING_STOCK`,`RECEIPTS`,`ISSUES`
        FROM `warehouse_stock`.`item_stock`
        WHERE `ITEM_NAME` = 'Pencil Box' 
            AND STOCK_DATE = '{start_date}'
        """ )

print("Data in local databases : \n------------------------------------")
for warehouse_id, opening_stock, receipts, issues in local_check_cursor:
    print(warehouse_id, opening_stock, receipts, issues)

#Query global database for summary records
global_check_cursor=global_db_conn.cursor()
global_check_cursor.execute(
    f"""SELECT `OPENING_STOCK`,`RECEIPTS`,`ISSUES`
        FROM `global_stock`.`item_stock`
        WHERE `ITEM_NAME` = 'Pencil Box' 
            AND STOCK_DATE = '{start_date}'
        """ )

print("\nData in global database : \n------------------------------------")
for opening_stock, receipts, issues in global_check_cursor:
    print( opening_stock, receipts, issues)


Data in local databases : 
------------------------------------
NewYork 61 17 41
LosAngeles 10 38 47
London 47 48 89

Data in global database : 
------------------------------------
118 103 177
